## 작업형 2 예시
- 데이터 가져오기
    + [고객지원] - [공지사항] - 제2회 빅데이터분석기사 실기 안내(수정: 체험링크 추가)

## 문제
- 아래는 백화점 고객의 1년 간 구매 데이터이다.
- 제공 데이터 목록
  + y_train.csv : 고객의 성별 데이터(학습용, CSV 형식의 파일)
  + X_train.csv, X_test.csv : 고객의 상품구매 속성(학습용 및 평가용), CSV 형식의 파일
- 데이터 형식 및 내용
  + y_train.csv (3,500명 데이터)
    * custid : 고객 ID
    * gender: 고객의 성별(0: 여자, 1:남자)
- 고객 3,5000명에 대한 학습용 데이터를 이용하여 성별예측 모형을 만드 후, 이를 평가용 데이터에 적용하여 얻은 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성
- 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

In [18]:
import pandas as pd 
pd.DataFrame({
    'custid' : [3500, 3501, 3502], 
    'gender' : [0.267, 0.578, 0.885]
})

,custid,gender
0,3500,0.267
1,3501,0.578
2,3502,0.885


In [5]:
import pandas as pd

DATA_PATH = 'data/department_store/'

X_test = pd.read_csv(DATA_PATH + "X_test.csv", encoding="euc-kr")   # 작업장 환경에서는 불필요
X_train = pd.read_csv(DATA_PATH + "X_train.csv", encoding="euc-kr")
y_train = pd.read_csv(DATA_PATH + "y_train.csv", encoding="euc-kr")

In [6]:
X_test.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8


In [8]:
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [9]:
y_train.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


## step 01. 데이터 처리

In [11]:
X_train.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [13]:
X_test.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [15]:
y_train.isnull().sum()

cust_id    0
gender     0
dtype: int64

- 결측치 처리 할 것
- 환불금액이기 때문에 평균, 중간값 이런 것보다는 0원으로 처리하는 것이 합리적임. 

In [16]:
X_train['환불금액'] = X_train['환불금액'].fillna(0)
X_train.isnull().sum()

cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64

In [17]:
X_test['환불금액'] = X_test['환불금액'].fillna(0)
X_test.isnull().sum()

cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64

## 데이터 확인

In [24]:
X_train['주구매상품'].value_counts()

기타        595
가공식품      546
농산물       339
화장품       264
시티웨어      213
디자이너      193
수산품       153
캐주얼       101
명품        100
섬유잡화       98
골프         82
스포츠        69
일용잡화       64
모피/피혁      57
육류         57
남성 캐주얼     55
구두         54
건강식품       47
차/커피       44
피혁잡화       40
아동         40
축산가공       35
주방용품       32
셔츠         30
젓갈/반찬      29
주방가전       26
트래디셔널      23
남성정장       22
생활잡화       15
주류         14
가구         10
커리어         9
대형가전        8
란제리/내의      8
식기          7
액세서리        5
침구/수예       4
통신/컴퓨터      3
보석          3
남성 트랜디      2
소형가전        2
악기          2
Name: 주구매상품, dtype: int64

In [25]:
X_test['주구매상품'].value_counts()

기타        465
가공식품      395
농산물       235
화장품       177
시티웨어      168
디자이너      123
수산품        86
섬유잡화       72
캐주얼        65
명품         60
차/커피       55
골프         47
모피/피혁      47
남성 캐주얼     46
구두         44
스포츠        43
육류         38
건강식품       36
일용잡화       30
축산가공       28
남성정장       25
셔츠         25
아동         24
주방용품       21
젓갈/반찬      18
피혁잡화       16
주방가전       10
주류         10
란제리/내의     10
트래디셔널       9
식기          8
생활잡화        8
가구          7
액세서리        7
커리어         6
대형가전        6
남성 트랜디      4
통신/컴퓨터      3
침구/수예       3
보석          1
악기          1
Name: 주구매상품, dtype: int64

In [20]:
X_train['주구매지점'].value_counts()

본  점     1077
잠실점       474
분당점       436
부산본점      245
영등포점      241
일산점       198
강남점       145
광주점       114
노원점        90
청량리점       86
대전점        70
미아점        69
부평점        57
동래점        49
관악점        46
인천점        34
안양점        29
포항점        11
대구점         7
센텀시티점       6
울산점         6
전주점         5
창원점         4
상인점         1
Name: 주구매지점, dtype: int64

In [26]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     3500 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [28]:
list(X_train['주구매상품'].value_counts().index)

['기타',
 '가공식품',
 '농산물',
 '화장품',
 '시티웨어',
 '디자이너',
 '수산품',
 '캐주얼',
 '명품',
 '섬유잡화',
 '골프',
 '스포츠',
 '일용잡화',
 '모피/피혁',
 '육류',
 '남성 캐주얼',
 '구두',
 '건강식품',
 '차/커피',
 '피혁잡화',
 '아동',
 '축산가공',
 '주방용품',
 '셔츠',
 '젓갈/반찬',
 '주방가전',
 '트래디셔널',
 '남성정장',
 '생활잡화',
 '주류',
 '가구',
 '커리어',
 '대형가전',
 '란제리/내의',
 '식기',
 '액세서리',
 '침구/수예',
 '통신/컴퓨터',
 '보석',
 '남성 트랜디',
 '소형가전',
 '악기']

## ID값 제거

In [35]:
X_train = X_train.drop(['cust_id'], axis = 1)
cust_id = X_test.pop('cust_id')

In [37]:
X_train.head(1)

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17


In [38]:
X_test.head(1)

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26


## 모델 학습 (Pipeline 사용)

In [32]:
from sklearn import set_config

set_config(display='diagram')

In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

column_transformer = ColumnTransformer([
    ("robust_scaler", RobustScaler(), ["총구매액", "최대구매액", "환불금액", "내점일수", "내점당구매건수", "주말방문비율", "구매주기"]),
    ("cut_encoder", OrdinalEncoder(categories=[list(X_train['주구매상품'].value_counts().index)]), ["주구매상품"]),
    ("clarity_encoder", OrdinalEncoder(categories=[list(X_train['주구매지점'].value_counts().index)]), ["주구매지점"]),
], remainder="passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer),
    ("regressor", LGBMClassifier(random_state=42))
])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robust_scaler',
                                                  RobustScaler(),
                                                  ['총구매액', '최대구매액', '환불금액',
                                                   '내점일수', '내점당구매건수', '주말방문비율',
                                                   '구매주기']),
                                                 ('cut_encoder',
                                                  OrdinalEncoder(categories=[['기타',
                                                                              '가공식품',
                                                                              '농산물',
                                                                              '화장품',
                                                                              '시티웨어',
                                                                              '디자이너',
                                                                              '수산품',
                                                                              '캐주얼',
                                                                              '명품',
                                                                              '섬유잡화',
                                                                              '골프',
                                                                              '스포츠',
                                                                              '일용잡화',
                                                                              '모피/피혁',
                                                                              '육류',
                                                                              '남성 '
                                                                              '캐주얼',
                                                                              '구두',
                                                                              '건강식품',
                                                                              '차/커피',
                                                                              '피혁잡화',
                                                                              '아동',
                                                                              '축산가공',
                                                                              '주방용품',
                                                                              '셔츠',
                                                                              '젓갈/반찬',
                                                                              '주방가전',
                                                                              '트래디셔널',
                                                                              '남성정장',
                                                                              '생활잡화',
                                                                              '주류', ...]]),
                                                  ['주구매상품']),
                                                 ('clarity_encoder',
                                                  OrdinalEncoder(categories=[['본  '
                                                                              '점',
                                                                              '잠실점',
                                                                              '분당점',
                                                                              '부산본점',
                                                                              '영등포점',
                                                                              '일산점',
                                                                              '강남점',
                             

In [73]:
import numpy as np 

pipeline.fit(X_train, y_train['gender'].values)

C:\Users\j2hoo\OneDrive\Documents\2023\ds-certification\venv\lib\site-packages\sklearn\compose\_column_transformer.py:610: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  self._df_columns[remainder_columns].tolist()


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robust_scaler',
                                                  RobustScaler(),
                                                  ['총구매액', '최대구매액', '환불금액',
                                                   '내점일수', '내점당구매건수', '주말방문비율',
                                                   '구매주기']),
                                                 ('cut_encoder',
                                                  OrdinalEncoder(categories=[['기타',
                                                                              '가공식품',
                                                                              '농산물',
                                                                              '화장품',
                                                                              '시티웨어',
                                                                              '디자이너',
                                                                              '수산품',
                                                                              '캐주얼',
                                                                              '명품',
                                                                              '섬유잡화',
                                                                              '골프',
                                                                              '스포츠',
                                                                              '일용잡화',
                                                                              '모피/피혁',
                                                                              '육류',
                                                                              '남성 '
                                                                              '캐주얼',
                                                                              '구두',
                                                                              '건강식품',
                                                                              '차/커피',
                                                                              '피혁잡화',
                                                                              '아동',
                                                                              '축산가공',
                                                                              '주방용품',
                                                                              '셔츠',
                                                                              '젓갈/반찬',
                                                                              '주방가전',
                                                                              '트래디셔널',
                                                                              '남성정장',
                                                                              '생활잡화',
                                                                              '주류', ...]]),
                                                  ['주구매상품']),
                                                 ('clarity_encoder',
                                                  OrdinalEncoder(categories=[['본  '
                                                                              '점',
                                                                              '잠실점',
                                                                              '분당점',
                                                                              '부산본점',
                                                                              '영등포점',
                                                                              '일산점',
                                                                              '강남점',
                             

## 모형 평가
- 정확도를 계산한다. 

In [74]:
pipeline.score(X_train, y_train['gender'])

0.8897142857142857

In [75]:
predictions = pipeline.predict_proba(X_test)
predictions[:, 1]

array([0.48938383, 0.09836715, 0.09363401, ..., 0.34992229, 0.61846457,
       0.63511339])

## 제출

In [76]:
pd.DataFrame({'cust_id': cust_id, 'gender': predictions[:, 1]}).to_csv('003000000.csv', index=False)